# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants
load_dotenv(override=True)

True

In [3]:
# Initializing GPT

MODEL_GPT = os.getenv("MODEL_GPT") 

In [4]:
# Initializing LLAMA 

MODEL_LLAMA = os.getenv("OLLAMA_MODEL")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL") 
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY")

ollama_via_openai = OpenAI(base_url=OLLAMA_BASE_URL, api_key=OLLAMA_API_KEY)
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
# Set up the environment

api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [6]:
# Create the system prompt
system_prompt = "You are a coding assistant the answers questions about small code snippets in a \
very insightful and illustrative way, giving examples and explaining how these concepts are embedded \
in a broader context. Stay away from 'talking' to the user, just inform them like a wikipedia article. \
Create the output as Markdown. Whenever possible, show examples in 'code' formatted boxes."
system_prompt += """
An example for this last requirement would be: \
```python
    def example_function():
```
"""

In [7]:
# here is the question; type over this to ask something new

example_snippet = 'yield from {book.get("author") for book in books if book.get("author")}'

def get_user_prompt(snippet):
    user_prompt = "Please explain what this code does and why:\n\n"
    user_prompt += snippet
    user_prompt += "\n"
    return user_prompt
print(get_user_prompt(example_snippet))


Please explain what this code does and why:

yield from {book.get("author") for book in books if book.get("author")}



In [ ]:
def handle_stream(stream_response):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream_response:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [9]:
def create_model_handler(model):
    def coding_answer(code_snippet, system_prompt):
        user_prompt = get_user_prompt(code_snippet)
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
        if model.startswith("llama"):
            # Handle Ollama models
            response = ollama_via_openai.chat.completions.create(
                model=model,
                messages=messages
            )
            return display(Markdown(response.choices[0].message.content))
        else:
            # Handle OpenAI models
            stream = openai.chat.completions.create(
                model=model,
                messages=messages,
                stream=True
            )
            return handle_stream(stream)
    
    return coding_answer

In [10]:
ollama_coder = create_model_handler(MODEL_LLAMA)
openai_coder = create_model_handler(MODEL_GPT)

In [11]:
# Get Llama 3.2 to answer
ollama_response = ollama_coder(example_snippet, system_prompt)

**Example Code Explanation**

```python
def author_generator(books):
    yield from {book["author"] for book in books if "author" in book}
```

**Purpose**
--------

The code snippet defines a generator function called `author_generator` that yields the authors of books stored in a list (`books`). The authors are extracted based on the presence of the key `"author"` within each book dictionary.

**Code Breakdown**

*   `yield from`: This keyword is used to delegate iteration to another iterable (in this case, a dictionary comprehension). It allows us to produce a collection of results without loading them all into memory at once.
*   `{... for book in books if book.get("author")}`: A dictionary comprehension that iterates over the list of books (`books`). For each book:
    *   `if book.get("author")`: This condition filters out books that do not have an `"author"` key. The `get()` method returns a value or `None` if the key does not exist, preventing a KeyError.
    *   `{book["author"]}`: If the book has an `"author"` key, this expression extracts its value.

**How it Works**

1.  Initialize an empty list (`books`) that contains dictionaries representing books with their respective authors.
2.  Call the `author_generator` function and iterate over the resulting iterable of authors using a for-each loop or another context manager.
3.  The generator will produce one author at a time, yielding it to the caller.

**Example Use Case**

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2"},  # Missing author key
    {"title": "Book 3", "author": "Author C"}
]

for author in author_generator(books):
    print(author)
# Output:
# Author A
# Author C
```

In this example, the generator function `author_generator` is used to extract and yield only the authors of books that have an `"author"` key. The resulting iterable is iterated over using a for-each loop, printing each author as it becomes available.

In [14]:
# Get gpt-4o-mini to answer, with streaming
openai_response = openai_coder(example_snippet, system_prompt)

The provided code snippet utilizes a combination of the `yield from` statement, a set comprehension, and conditional filtering. Here's a breakdown of its components and their purpose:

### Breakdown of the Code

1. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}` is a set comprehension that iterates over a collection named `books`.
   - Each `book` is expected to be a dictionary, and it attempts to retrieve the value associated with the key `"author"` using the `get` method. 
   - The condition `if book.get("author")` ensures that only books with a non-`None` or non-empty string author will be included in the resulting set. This prevents `None` or empty strings from being added.

   #### Example:
   Given the following list of books:
   ```python
   books = [
       {"title": "Book A", "author": "Author 1"},
       {"title": "Book B", "author": None},
       {"title": "Book C", "author": "Author 2"},
       {"title": "Book D", "author": ""},
       {"title": "Book E"}
   ]
   ```
   The set comprehension would result in:
   ```python
   {"Author 1", "Author 2"}
   ```

2. **`yield from` Statement**:
   - `yield from <iterable>` is used within a generator function to yield all the values from the specified iterable (`{...}` in this case).
   - This statement simplifies the process of yielding multiple values from another iterable, making the generator code cleaner and more readable.

3. **Overall Purpose**:
   - The entire expression `yield from {book.get("author") for book in books if book.get("author")}` is likely located inside a generator function. Its main purpose is to yield each unique author from the provided collection of books, while ignoring any entries that do not have a valid author.

### Contextual Usage
This kind of code is particularly useful in situations where a large collection of records needs to be processed, particularly in applications dealing with data extraction or manipulation, such as:

- **Data Analysis**: Extracting unique authors from a dataset before analyzing their publications.
- **APIs**: Collecting unique author information to return in a response.
- **Text Processing**: Compiling lists of contributors to a collection of works.

### Conclusion
The code snippet effectively filters and extracts unique author names from a given list of book dictionaries, leveraging Python's capabilities for clean and efficient data processing through set comprehensions and generator functions.